In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 87% 105M/121M [00:02<00:00, 48.3MB/s] 
100% 121M/121M [00:02<00:00, 52.8MB/s]


In [ ]:
!unzip /content/face-expression-recognition-dataset.zip

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras_preprocessing.image import ImageDataGenerator
import cv2
import pandas as pd

In [ ]:
train_dir='/content/images/train'
val_dir='/content/images/validation'
img_height=160
batch_size = 32

In [ ]:
IMG_SIZE = (224, 224)
core_idg = ImageDataGenerator(rescale=(1/255.),
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.05, 
                              width_shift_range=0.1, 
                              rotation_range=5, 
                              shear_range = 0.1,
                              fill_mode = 'reflect'
                            )


In [ ]:
train_ds = core_idg.flow_from_directory(
    train_dir,
    target_size=(160,160),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=128,
    seed=11,
    )
valid_ds = core_idg.flow_from_directory(
    val_dir,
    target_size=(160,160),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=128,
    )

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(160,160,1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(7, activation='softmax')])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate	= 0.01), 
                         loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds,
                    verbose=1, epochs=10,validation_data=valid_ds)

Epoch 1/10
226/226 [==============================] - 107s 470ms/step - loss: 1.1766 - accuracy: 0.5545 - val_loss: 1.5640 - val_accuracy: 0.5650
Epoch 2/10
226/226 [==============================] - 104s 458ms/step - loss: 1.1433 - accuracy: 0.5688 - val_loss: 1.1413 - val_accuracy: 0.5655
Epoch 3/10
226/226 [==============================] - 106s 468ms/step - loss: 1.1314 - accuracy: 0.5741 - val_loss: 1.1527 - val_accuracy: 0.5698
Epoch 4/10
226/226 [==============================] - 102s 452ms/step - loss: 1.2179 - accuracy: 0.5360 - val_loss: 4.7305 - val_accuracy: 0.2086
Epoch 5/10
226/226 [==============================] - 102s 450ms/step - loss: 1.6234 - accuracy: 0.3531 - val_loss: 1.5356 - val_accuracy: 0.3963
Epoch 6/10
226/226 [==============================] - 102s 451ms/step - loss: 1.4827 - accuracy: 0.4239 - val_loss: 1.6176 - val_accuracy: 0.4120
Epoch 7/10
226/226 [==============================] - 102s 451ms/step - loss: 1.3990 - accuracy: 0.4604 - val_loss: 1.4318 -

In [ ]:
def create_model(base_model):
    base_model.trainable = False
    global_average_layer = tf.keras.layers.GlobalMaxPooling2D()(base_model.output)
    x4 = tf.keras.layers.Dense(512, activation='relu')(global_average_layer)
    x5 = tf.keras.layers.BatchNormalization()(x4)
    x6 = tf.keras.layers.Dropout(0.5)(x5)
    prediction_layer = tf.keras.layers.Dense(7, activation='softmax')(x6)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
    
    return model

In [ ]:
base_model4 = tf.keras.applications.Xception(input_shape=(224, 224,3),include_top=False, weights="imagenet")
model = create_model(base_model4)
model.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=['accuracy'])

In [ ]:
hist_x = model.fit(train_ds, epochs=10,
                    validation_data = valid_ds, 
                    verbose = 1)

Epoch 1/10
226/226 [==============================] - 400s 2s/step - loss: 1.7901 - accuracy: 0.3818 - val_loss: 1.4886 - val_accuracy: 0.4369
Epoch 2/10
226/226 [==============================] - 393s 2s/step - loss: 1.4946 - accuracy: 0.4385 - val_loss: 1.4211 - val_accuracy: 0.4629
Epoch 3/10
226/226 [==============================] - 393s 2s/step - loss: 1.4383 - accuracy: 0.4536 - val_loss: 1.4864 - val_accuracy: 0.4403
Epoch 4/10
226/226 [==============================] - 392s 2s/step - loss: 1.4147 - accuracy: 0.4660 - val_loss: 1.3844 - val_accuracy: 0.4707
Epoch 5/10
226/226 [==============================] - 393s 2s/step - loss: 1.3974 - accuracy: 0.4680 - val_loss: 1.3791 - val_accuracy: 0.4745
Epoch 6/10
226/226 [==============================] - 391s 2s/step - loss: 1.3921 - accuracy: 0.4727 - val_loss: 1.6121 - val_accuracy: 0.4172
Epoch 7/10
226/226 [==============================] - 391s 2s/step - loss: 1.3866 - accuracy: 0.4735 - val_loss: 1.3920 - val_accuracy: 0.4714